In [7]:
!pip install langchain_community 


  Using cached langchain_community-0.2.1-py3-none-any.whl.metadata (8.9 kB)
  Using cached PyYAML-6.0.1-cp310-cp310-win_amd64.whl.metadata (2.1 kB)
  Using cached SQLAlchemy-2.0.30-cp310-cp310-win_amd64.whl.metadata (9.8 kB)
  Using cached aiohttp-3.9.5-cp310-cp310-win_amd64.whl.metadata (7.7 kB)
  Using cached dataclasses_json-0.6.6-py3-none-any.whl.metadata (25 kB)
  Using cached langchain-0.2.1-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_core-0.2.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.1.67-py3-none-any.whl.metadata (13 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached tenacity-8.3.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached frozenlist-1.4.1-cp310-cp310-win_amd64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp310-cp310-win_amd64.whl.metadata (4.3 kB)
  Using cached yar

In [6]:
from flask import Flask, request, jsonify, render_template, session
import pandas as pd
import json
from langchain_community.document_loaders import CSVLoader


from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq
# from langchain_community.llms import Ollama
import functools

ModuleNotFoundError: No module named 'langchain_community'

In [3]:
import pandas as pd
df = pd.read_json("DATA/categorized_data.json")
df.head()

,heading,paragraph
0,Privacy policy,Information on data protection and our privacy...
1,Analytics,"Matomo, Google Analytics, Facebook Ads convers..."
2,Analytics,"Ads conversion tracking, Hotjar Form Analysis ..."
3,Analytics,Demandbase conversion tracking Personal Data: ...
4,Analytics,pixel) Personal Data: device information; Trac...


In [13]:
import os
from pinecone import Pinecone ,ServerlessSpec

pc = Pinecone(api_key="33fd2367-2090-4ff2-b22f-cc2503c2e1ee")

c:\Users\wwwya\OneDrive\Desktop\finetune_llama3\.venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [14]:
pc.list_indexes()

{'indexes': [{'dimension': 768,
              'host': 'eurotechuser-4nkv3rd.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'eurotechuser',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [15]:
index_name = "eurotechuser"
index = pc.Index(index_name)



In [10]:
index_name = "eurotechuser"
index = pc.Index(index_name)

In [2]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [2]:
pip uninstall txtai


In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("feature-extraction", model="jinaai/jina-embeddings-v2-base-zh", trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- configuration_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/jinaai/jina-bert-implementation:
- modeling_bert.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [3]:
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer
retrever = SentenceTransformer("flax-sentence-embeddings/all_datasets_v3_mpnet-base",device=device)

c:\Users\wwwya\OneDrive\Desktop\finetune_llama3\.venv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [13]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [14]:
retrever.get_sentence_embedding_dimension()

768

In [15]:
from tqdm.auto import tqdm

batch_size = 64
for i in tqdm(range(0,len(df),batch_size)):
    i_end = min(i+batch_size,len(df))
    batch = df.iloc[i:i_end]
    emb = retrever.encode(batch['paragraph'].tolist()).tolist()
    meta = batch.to_dict(orient='records')
    ids = [f"{idx}" for  idx in range(i,i_end)]

    up = list(zip(ids,emb,meta))
    _ = index.upsert(vectors=up)

index.describe_index_stats()

 33%|███▎      | 3/9 [00:16<00:33,  5.55s/it]


KeyboardInterrupt: 

In [ ]:
index.describe_index_stats()


{'dimension': 768,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 567}},
 'total_vector_count': 567}

In [16]:
df.iloc[:5].to_dict(orient='records')


[{'heading': 'Privacy policy',
  'paragraph': 'Information on data protection and our privacy policy'},
 {'heading': 'Analytics',
  'paragraph': 'Matomo, Google Analytics, Facebook Ads conversion tracking (Facebook pixel), Meta Events Manager, Google'},
 {'heading': 'Analytics',
  'paragraph': 'Ads conversion tracking, Hotjar Form Analysis & Conversion Funnels, Twitter Ads conversion tracking and'},
 {'heading': 'Analytics',
  'paragraph': 'Demandbase conversion tracking Personal Data: Trackers; Usage Data Outbrain conversion tracking (Outbrain'},
 {'heading': 'Analytics',
  'paragraph': 'pixel) Personal Data: device information; Trackers; Universally unique identifier (UUID); Usage Data'}]

In [16]:
def query_pinecone(query,top_k):
    xq = retrever.encode([query]).tolist()
    xc = index.query(vector=xq, top_k=top_k, include_metadata=True)
    return xc
def format_query(query,context):
    context = [f"<P> {m['metadata']['paragraph']}" for m in context]
    context = " ".join(context)
    query =  f"Querstion: {query} context: {context}"
    return query

In [17]:
query = "i want to know about the policys of eurotech"
result = query_pinecone(query,top_k=3)
result

NameError: name 'retrever' is not defined

In [19]:
from pprint import pprint

query = format_query(query=query,context= result['matches'])
pprint(query)

('Querstion: i want to know about the policys of eurotech context: <P> If you '
 "have any questions about this policy or Eurotech Xchange's privacy "
 "practices, please contact Eurotech Xchange. <P> on Eurotech Xchange's "
 'website to other personal information you may provide to Eurotech Xchange at '
 'the <P> information about you as described in this policy, which Eurotech '
 'Xchange handles as stated in this policy.')


In [20]:
query

"Querstion: i want to know about the policys of eurotech context: <P> If you have any questions about this policy or Eurotech Xchange's privacy practices, please contact Eurotech Xchange. <P> on Eurotech Xchange's website to other personal information you may provide to Eurotech Xchange at the <P> information about you as described in this policy, which Eurotech Xchange handles as stated in this policy."

In [6]:
!pip install vector_quantize_pytorch

   ---------------------------------------- 0.0/101.6 kB ? eta -:--:--
   ------------------------ --------------- 61.4/101.6 kB 1.7 MB/s eta 0:00:01
   ---------------------------------------- 101.6/101.6 kB 1.5 MB/s eta 0:00:00


In [12]:
import torch
import ChatTTS
from IPython.display import Audio

# Initialize ChatTTS
chat = ChatTTS.Chat()
chat.load_models()
chat.load_models()
print("Models loaded:", chat.check_model()) 

# # Define the text to be converted to speech
# texts = ["Hello, welcome to ChatTTS!",]

# # Generate speech
# wavs = chat.infer(texts, use_decoder=True)

# # Play the generated audio
# Audio(wavs[0], rate=24_000, autoplay=True)

INFO:ChatTTS.core:use cpu
INFO:ChatTTS.core:use cpu


Models loaded: False
